Importação de Libs

In [ ]:
!pip install -q google-adk

In [ ]:
import os
import warnings
from datetime import date
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from google.colab import userdata

warnings.filterwarnings("ignore")


In [ ]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    # O Runner pode gerenciar a criação da sessão se ela não existir.
    session_service.create_session(
        app_name=agent.name, user_id="user1", session_id="session1"
    )
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response.strip()

In [ ]:
def agente_previsao_chuva(cidade: str, data_alvo: str) -> str:
    instruction_text = f"""
    Você é um assistente meteorológico preciso que utiliza a ferramenta Google Search.
    Sua tarefa é buscar a previsão de chuva para a cidade e data fornecidas.
    Organize a resposta estritamente nos seguintes períodos, indicando a intensidade da chuva (leve, moderada, forte, muito forte) ou "sem chuva":
    - Manhã (06:00-12:00)
    - Tarde (12:00-18:00)
    - Noite (18:00-00:00)
    - Madrugada (00:00-06:00)

    Se não houver dados para um período específico, informe "sem informação".

    Exemplo de Resposta:
    Manhã: sem chuva
    Tarde: chuva forte
    Noite: chuva moderada
    Madrugada: sem informação

    Dados para a consulta:
    Cidade: {cidade}
    Data: {data_alvo}
    """
    agente = Agent(
        name="agente_previsao_chuva",
        model=MODEL_NAME,
        instruction=instruction_text,
        description="Agente para buscar previsão de chuva (horários e intensidade) via Google Search.",
        tools=[google_search]
    )
    entrada = f"Qual a previsão de chuva para {cidade} em {data_alvo}, detalhada por período (manhã, tarde, noite, madrugada) e intensidade?"
    return call_agent(agente, entrada)

In [ ]:
def agente_tabua_mare(local: str, data_alvo: str) -> str:
    instruction_text = f"""
    Você é um especialista em marés e utiliza a ferramenta Google Search.
    Sua tarefa é fornecer a tábua da maré (horários de maré alta e baixa, com alturas em metros) para o local e data informados.

    Formato da Resposta:
    Maré alta: HH:MM (X.Ym), HH:MM (X.Ym)
    Maré baixa: HH:MM (X.Ym), HH:MM (X.Ym)

    Se alguma informação (horário ou altura) não estiver disponível, indique "sem informação" para o item específico.
    Se não houver dados para maré alta ou baixa como um todo, informe "Maré alta: sem informação" ou "Maré baixa: sem informação".

    Dados para a consulta:
    Local: {local}
    Data: {data_alvo}
    """
    agente = Agent(
        name="agente_tabua_mare",
        model=MODEL_NAME,
        instruction=instruction_text,
        description="Agente para buscar a tábua da maré (horários e alturas) via Google Search.",
        tools=[google_search]
    )
    entrada = f"Qual a tábua da maré para {local} em {data_alvo}, incluindo horários e alturas das marés alta e baixa?"
    return call_agent(agente, entrada)

In [ ]:
def agente_pontos_criticos_alagamento(cidade: str, caminho_arquivo_txt: str) -> str:
    pontos_locais = []
    try:
        with open(caminho_arquivo_txt, 'r', encoding='utf-8') as f:
            for linha in f:
                linha_strip = linha.strip()
                if linha_strip:
                    pontos_locais.append(linha_strip)
    except FileNotFoundError:
        print(f"AVISO: Arquivo de pontos críticos '{caminho_arquivo_txt}' não encontrado. Buscando apenas online e dados fixos.")

    exemplos_oficiais_texto = (
        "Considere também os seguintes exemplos de pontos críticos de referência e seus riscos associados:\n"
        "- Passagem Tradição - Comunidade Fé em Deus II (Bairro São João do Outeiro): risco ALTO de inundação, ~150 famílias afetadas.\n"
        "- Av. Beira Mar - Ilha do Outeiro: risco MUITO ALTO de erosão costeira.\n"
        "- Praia Belo Paraíso (Bairro Fama - Ilha do Outeiro): risco MUITO ALTO de erosão costeira e inundações.\n"
        "- Distrito de Icoaraci (bairros Agulha, Ponta Grossa, Paracuri, Baixada Fluminense): risco MUITO ALTO de alagamento.\n"
        "- Áreas como Paracuri, Parque Guajará, Conj. Tocantins, Vila Riso, Buraco Fundo, Canarinho, Piçarreira: risco de inundação recorrente (Fev-Maio)."
    )

    texto_pontos_locais_formatado = ""
    if pontos_locais:
        texto_pontos_locais_formatado = (
            "Adicionalmente, foram registrados os seguintes pontos críticos localmente (a partir do arquivo fornecido):\n"
            + "\n".join(f"- {p}" for p in pontos_locais)
        )

    instruction_text = f"""
    Você é um assistente de pesquisa focado em identificar áreas de risco de alagamento em {cidade}, utilizando Google Search e dados fornecidos.
    Sua tarefa é consolidar uma lista de bairros, comunidades ou pontos críticos suscetíveis a alagamentos.
    Priorize informações de fontes oficiais (prefeitura, defesa civil) obtidas pela busca.

    {exemplos_oficiais_texto}
    {texto_pontos_locais_formatado if texto_pontos_locais_formatado else "Nenhum ponto crítico adicional foi fornecido de arquivos locais."}

    Instruções para a Resposta:
    1. Use o Google Search para encontrar a lista mais atualizada de pontos críticos em {cidade} de fontes oficiais.
    2. Complemente com as informações de referência e do arquivo local (se fornecidas acima).
    3. Liste SOMENTE os nomes dos bairros, comunidades ou locais críticos.
    4. Agrupe por área ou distrito, se a informação estiver clara nas fontes.
    5. NÃO repita nomes. Consolide as informações.
    6. Se dados sobre o número de pessoas/famílias afetadas ou o nível de risco (ex: alto, muito alto) estiverem explicitamente associados a um local nas fontes ou nos dados fornecidos, inclua essa informação entre parênteses.

    Exemplo de Formato de Resposta:
    Distrito de Icoaraci:
    - Agulha (risco MUITO ALTO)
    - Ponta Grossa
    - Paracuri (risco MUITO ALTO, inundação recorrente Fev-Maio)
    Bairro São João do Outeiro:
    - Passagem Tradição - Comunidade Fé em Deus II (risco ALTO, ~150 famílias afetadas)
    Outros Pontos (obtidos da busca ou arquivo):
    - Bairro do Umarizal
    - [Nome do Ponto do Arquivo Local 1]
    """
    agente = Agent(
        name="agente_pontos_criticos_alagamento",
        model=MODEL_NAME,
        instruction=instruction_text,
        description="Agente para identificar pontos críticos de alagamento, combinando busca online com dados fornecidos.",
        tools=[google_search]
    )
    entrada = f"Liste os bairros e pontos críticos de alagamento em {cidade}, com base em dados oficiais, estudos e registros fornecidos."
    return call_agent(agente, entrada)

In [ ]:
def agente_altitude_bairro(bairro: str, cidade: str) -> str:
    instruction_text = f"""
    Você é um assistente geográfico que utiliza Google Search.
    Sua tarefa é encontrar a altitude média aproximada do bairro {bairro} na cidade de {cidade}.
    Responda APENAS o valor em metros (ex: "8 metros").
    Se a informação não for encontrada de forma confiável, responda "sem informação".
    """
    agente = Agent(
        name="agente_altitude_bairro",
        model=MODEL_NAME,
        instruction=instruction_text,
        description="Agente para buscar a altitude média de um bairro via Google Search.",
        tools=[google_search]
    )
    entrada = f"Qual é a altitude média do bairro {bairro} em {cidade}?"
    return call_agent(agente, entrada)

In [ ]:
def agente_analise_risco_alagamento(
    bairro: str,
    cidade: str,
    previsao_chuva: str,
    tabua_mare: str,
    lista_pontos_criticos: str,
    altitude_bairro: str
) -> str:
    instruction_text = f"""
    Você é um analista de risco de alagamento.
    Sua função é avaliar o risco de alagamento para um bairro específico com base nos dados fornecidos.
    Classifique o risco como: BAIXO, MÉDIO, ALTO ou MUITO ALTO.
    Forneça uma justificativa concisa para sua avaliação, considerando a combinação dos fatores.

    Dados para Análise:
    - Bairro: {bairro}
    - Cidade: {cidade}
    - Previsão de Chuva (para a data relevante): {previsao_chuva}
    - Tábua da Maré (para a data relevante): {tabua_mare}
    - Lista de Pontos Críticos Conhecidos na Cidade (pode incluir o bairro ou áreas próximas): {lista_pontos_criticos}
    - Altitude Média do Bairro: {altitude_bairro}

    Diretrizes para Avaliação do Risco:
    - **MUITO ALTO RISCO**: Combinação crítica de fatores: chuva forte/muito forte, maré alta significativa (especialmente se coincidente com picos de chuva), bairro explicitamente listado como de alto/muito alto risco ou com histórico grave, e/ou altitude muito baixa.
    - **ALTO RISCO**: Presença de múltiplos fatores de risco: chuva moderada a forte, maré alta, bairro listado como ponto crítico (mesmo sem nível especificado) ou com baixa altitude.
    - **MÉDIO RISCO**: Alguns fatores de risco presentes: chuva leve a moderada, maré normal a alta (sem extremos), e/ou o bairro possui alguma vulnerabilidade conhecida (ex: altitude moderada, proximidade a áreas de risco).
    - **BAIXO RISCO**: Ausência de fatores de risco significativos: sem previsão de chuva expressiva, maré normal ou baixa, bairro não listado como crítico e/ou com altitude que não sugere vulnerabilidade.

    Considerações Adicionais:
    - Se a altitude do bairro for "sem informação", mencione isso na justificativa e baseie a análise nos outros fatores, indicando que a ausência dessa informação pode impactar a precisão.
    - Verifique se o bairro '{bairro}' está contido ou se as características descritas na 'Lista de Pontos Críticos Conhecidos na Cidade' se aplicam diretamente a ele.
    - A coincidência temporal de chuva intensa e maré alta eleva significativamente o risco.

    Formato da Resposta:
    Alerta: [NÍVEL DE RISCO] de alagamento no bairro {bairro}.
    Justificativa: [Explicação concisa baseada nos dados, diretrizes e considerações]

    Exemplo de Resposta:
    Alerta: ALTO RISCO de alagamento no bairro do Marco.
    Justificativa: Previsão de chuva forte para a tarde, coincidindo com maré alta de 3.2m. O bairro do Marco consta na lista de pontos críticos da cidade. A altitude de 8 metros é relativamente baixa, contribuindo para o risco.
    """
    agente = Agent(
        name="agente_analise_risco_alagamento",
        model=MODEL_NAME, # Para análise mais complexa, um modelo como gemini-1.5-pro-latest poderia ser considerado.
        instruction=instruction_text,
        description="Agente analítico para determinar o nível de risco de alagamento e justificar.",
        tools=[] # Este agente apenas analisa os dados fornecidos.
    )
    entrada = f"Com base nos dados fornecidos, qual o nível de risco de alagamento para o bairro {bairro} em {cidade} e por quê?"
    return call_agent(agente, entrada)

In [ ]:
MODEL_NAME = "gemini-2.0-flash"
# --- Parâmetros de Entrada ---
bairro_alvo = "Terra Firme"
cidade_alvo = "Belém, PA"
# Use date.today().isoformat() para a data atual ou especifique uma data.
# data_alvo_str = date.today().isoformat()
data_alvo_str = "2025-05-16" # Data do exemplo original

# Caminho para o arquivo de riscos geológicos. Ajuste se necessário.
caminho_arquivo_riscos = 'riscos-geologicos.txt'

# Cria um arquivo de exemplo se não existir, para evitar erro na primeira execução.
if not os.path.exists(caminho_arquivo_riscos):
    try:
        with open(caminho_arquivo_riscos, 'w', encoding='utf-8') as f:
            f.write("Exemplo de ponto de risco local 1 no bairro Y\n")
            f.write("Outra área de atenção na Travessa Z\n")
        print(f"INFO: Arquivo '{caminho_arquivo_riscos}' não encontrado. Um arquivo de exemplo foi criado.")
    except IOError as e:
        print(f"ERRO: Não foi possível criar o arquivo de exemplo '{caminho_arquivo_riscos}': {e}")
        # Decide se quer parar ou continuar sem o arquivo local
        # return

print(f"--- Iniciando análise de risco para o bairro {bairro_alvo} em {cidade_alvo} para {data_alvo_str} ---")

previsao_chuva_info = agente_previsao_chuva(cidade=cidade_alvo, data_alvo=data_alvo_str)
print(f"\n[INFO] Previsão de Chuva para {cidade_alvo} em {data_alvo_str}:\n{previsao_chuva_info}")

tabua_mare_info = agente_tabua_mare(local=cidade_alvo, data_alvo=data_alvo_str)
print(f"\n[INFO] Tábua da Maré para {cidade_alvo} em {data_alvo_str}:\n{tabua_mare_info}")

pontos_criticos_info = agente_pontos_criticos_alagamento(cidade=cidade_alvo, caminho_arquivo_txt=caminho_arquivo_riscos)
print(f"\n[INFO] Pontos Críticos de Alagamento em {cidade_alvo}:\n{pontos_criticos_info}")

altitude_bairro_info = agente_altitude_bairro(bairro=bairro_alvo, cidade=cidade_alvo)
print(f"\n[INFO] Altitude Média do Bairro {bairro_alvo}:\n{altitude_bairro_info}")

print("\n--- Gerando Alerta Final ---")
alerta_final = agente_analise_risco_alagamento(
    bairro=bairro_alvo,
    cidade=cidade_alvo,
    previsao_chuva=previsao_chuva_info,
    tabua_mare=tabua_mare_info,
    lista_pontos_criticos=pontos_criticos_info,
    altitude_bairro=altitude_bairro_info
)
print(f"\n[ALERTA FINAL] Análise de Risco para {bairro_alvo}, {cidade_alvo} em {data_alvo_str}:\n{alerta_final}")


--- Iniciando análise de risco para o bairro Terra Firme em Belém, PA para 2025-05-16 ---

[INFO] Previsão de Chuva para Belém, PA em 2025-05-16:
Com base nas informações disponíveis para o dia 16 de maio de 2025 em Belém, PA:

*   **Manhã (06:00-12:00):** Sol com aumento de nuvens e pancadas de chuva e trovoadas. Parcialmente nublado a poucas nuvens. Indicativo de chuvas isoladas na madrugada. Chuvoso de manhã.
*   **Tarde (12:00-18:00):** Chuvas leves a moderadas com trovoadas isoladas. Sol com aumento de nuvens e pancadas de chuva e trovoadas. Parcialmente nublado a nublado.
*   **Noite (18:00-00:00):** Chuvas rápidas no início do período. Parcialmente nublado a poucas nuvens. Temporal. Noite chuvosa.
*   **Madrugada (00:00-06:00):** Indicativo de chuvas isoladas. Chuvoso. Sol e chuva.

[INFO] Tábua da Maré para Belém, PA em 2025-05-16:
Com base nos resultados da pesquisa, a tábua de marés para Belém, PA em 2025-05-16 é a seguinte:

Maré alta: 00:45 (2.98m), 12:58 (3.32m)
Maré baixa